Creating basic API call

In [1]:
# importing requisite libraries
import requests
from dotenv import load_dotenv
import os 
import pandas as pd
import numpy as np
import alpaca_trade_api as tradeapi
import json

In [2]:
load_dotenv()

Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 8


True

In [3]:
# Bringing in alpaca api keys
alpaca_api_key = os.getenv("APCA_API_KEY_ID")
alpaca_secret_key = os.getenv("APCA_API_SECRET_KEY")
# Create the Alpaca API object
# YOUR CODE HERE!
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version= "v2")

In [4]:
# testing alpaca keys
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpca Secret Key type: <class 'str'>


In [10]:
# Defining timeframe and ticker parameters for my API pull
current_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()

ticker = ["JPM"]

timeframe = "15Min"

In [11]:
# Creating dataframe with every feature on Alpaca's get_barset
df_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    start=current_date
).df

df_ticker.head()

JPM                                 
                              open    high      low    close volume
2020-01-02 09:30:00-05:00  140.070  140.14  139.520  139.670  29953
2020-01-02 09:45:00-05:00  139.675  139.91  139.440  139.670  24343
2020-01-02 10:00:00-05:00  139.635  139.81  139.520  139.640  19021
2020-01-02 10:15:00-05:00  139.600  139.60  139.340  139.400  18856
2020-01-02 10:30:00-05:00  139.460  139.54  139.280  139.535  19695
...                            ...     ...      ...      ...    ...
2021-01-06 12:15:00-05:00  131.670  131.96  131.665  131.960  22992
2021-01-06 12:30:00-05:00  131.960  132.30  131.920  132.280  17191
2021-01-06 12:45:00-05:00  132.275  132.58  131.960  132.580  20470
2021-01-06 13:00:00-05:00  132.615  132.65  132.265  132.360  21793
2021-01-06 13:15:00-05:00  132.350  132.55  132.350  132.460   3561

[6857 rows x 5 columns]

In [17]:
# Creating a clsing price dataframe. Had to get a little weird with this code. 
closing_price = pd.DataFrame()

for time_period in ticker:
    closing_price[x] = df_ticker[time_period]["close"]

closing_price.head()


,JPM
2020-01-02 09:30:00-05:00,139.670
2020-01-02 09:45:00-05:00,139.670
2020-01-02 10:00:00-05:00,139.640
2020-01-02 10:15:00-05:00,139.400
2020-01-02 10:30:00-05:00,139.535


Putting the API calll into a function

In [31]:
stock = ["TSLA"]
def get_dat_data(stock):
    
    
    alpaca_api_key = os.getenv("APCA_API_KEY_ID")
    alpaca_secret_key = os.getenv("APCA_API_SECRET_KEY")
    # Create the Alpaca API object
    # YOUR CODE HERE!
    alpaca = tradeapi.REST(
        alpaca_api_key,
        alpaca_secret_key,
        api_version= "v2")

    current_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()

    timeframe = "15Min"

    df_ticker = alpaca.get_barset(
        stock,
        timeframe,
        start=current_date
        ).df
    
    
    closing_price = pd.DataFrame()

    for time_period in ticker:
        closing_price[x] = df_ticker[time_period]["close"]

    closing_price.head()

In [33]:
get_dat_data(stock)

KeyError: 'T'